In [ ]:
# Import all the packages and libraries
from langchain.llms import CTransformers
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embdeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pinecone

In [ ]:
# Extracting Data from PDF
def load_pdf(data):
  loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
  documents = loader.load()
  return documents
extracted_data = load_pdf("data/")

In [ ]:
# Creating Text Chunks
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks
text_chunks = text_split(extracted_data)
print("Length of the chunks: ", len(text_chunks)) # 7020

In [ ]:
# Download the embedding model
def download_hugging_face_embeddings():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings
embeddings = download_hugging_face_embeddings()

In [ ]:
# Query Embedding:
query_result = embeddings.embed_query("Hello World")
print("Length: ", len(query_result)) # 384

In [ ]:
# Creating a Pinecone Cluster
# - Login into Pinecone: https://www.pinecone.io/ > Dashboard
# - API Keys > Create API Key and use it
# - Indexes > Create Index: medical-chatbot, 384 Dimensions, Cosine (HuggingFace Model: sentence-transformers/all-MiniLM-L6-v2) > Use the ENVIRONMENT

# Initializing the PineCone
PINECONE_API_KEY = "..."
PINECONE_API_ENV = "gcp-starter"
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
index_name="medical-chatbot"

In [ ]:
# Creating embeddings for each of the text chunks and storing it
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [ ]:
# - Goto the Pinecone website, and view all the vectors that are stored. Total: 7020
# - Now, performing semantic index: Doing Semantic or Similarity Search
# If we already have an index we can load it like this
docsearch = Pinecone.from_existing_index(index_name, embeddings)
query = "What are Allergies?"
docs = docsearch.similarity_search(query, k=3)
print("Result", docs)

In [ ]:
# Implementing the PromptTemplate
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else...
Helpful answer:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama", config={"max_new_tokens":512, "temperature":0.8})

In [ ]:
# Performing Retrieval Question-Answering
qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k':2}), return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

while True:
  user_input = input(f"Input Prompt: ")
  result = qa({"query": user_input})
  print("Response: ", result["result"])